In [3]:
import os
import sys
import torch
import datetime
import numpy as np
from utils import *
from model import *
from tqdm import tqdm
from time import time
from torch import Tensor, nn
from dataset_factory import *
import torch_optimizer as optim
import torch.distributed as dist
from dataclasses import dataclass
from accelerate import Accelerator
from transformers.file_utils import ModelOutput
from torch.utils.data import DataLoader, Dataset
from typing import List, Optional, Tuple, Union, Dict
from deepspeed.ops.adam import DeepSpeedCPUAdam, FusedAdam
from torch.optim.lr_scheduler import CosineAnnealingLR, LinearLR
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
os.environ['http_proxy'] = 'http://172.19.57.45:3128'
os.environ['https_proxy'] = 'http://172.19.57.45:3128'
sys.path.append('/root/paddlejob/workspace/env_run/ReGPT')
%cd /root/paddlejob/workspace/env_run/ReGPT
config = get_config('config/rellama_config.yaml')
ReGPT_kwargs = config['ReGPT_kwargs']
config['training'].update(ReGPT_kwargs)
config['dataset']['train'].update(ReGPT_kwargs)
config['dataset']['test'].update(ReGPT_kwargs)
generation_kwargs = config['generation_kwargs']
config['training'].update(generation_kwargs)
train_config = config['training']
dataset_config = config['dataset']
train_config['negative_depth'] = dataset_config['train']['negative_depth']
dataset_config['train']['predict_from_last'] = train_config['predict_from_last']
dataset_config['test']['predict_from_last'] = train_config['predict_from_last']

tokenizer = AutoTokenizer.from_pretrained(train_config['tokenizer_name_or_path'])
tokenizer.pad_token = tokenizer.eos_token
train_config['eos_token_id'] = tokenizer.eos_token_id

dataset_class = {"DialogSFTDataset": DialogSFTDataset, "CorpusPretrainDataset": CorpusPretrainDataset, "ReGPTDialogSFTDataset": ReGPTDialogSFTDataset, "ReGPTCorpusPretrainDataset": ReGPTCorpusPretrainDataset}

config['training']['model_name_or_path'] = 'output/SFT-best/'
print_args(config)
model = ReGPTForCausalLM(train_config)
# model.cuda()
# model = model.half()


def move_to_cuda(kwargs, device='cuda:0'):
    for key in kwargs:
        kwargs[key] = kwargs[key].to(device)
def generate(text='Tsinghua University is a '):
    text+=' '
    kwargs = tokenizer([text],return_tensors='pt')
#     move_to_cuda(kwargs)
    output_ids = model.generate(**kwargs)
#     print(tokenizer.convert_ids_to_tokens(output_ids.cpu().numpy().squeeze()))
    return tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(output_ids.cpu().numpy().squeeze()))

from time import time
start = time()
model.train_config['do_sample'] = True
model.train_config['top_k'] = 5
model.train_config['top_p'] = 0.8
model.train_config['max_length'] = 100
print(generate('Beethoven'))
print(time()-start)

[2023-12-01 01:01:05,575] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/root/paddlejob/workspace/env_run/ReGPT
trainer: ReGPTLanguageModelTrainer
training:
	project_name: llama2-7b-sft
	model_name_or_path: output/SFT-best/
	tokenizer_name_or_path: ../data_of_ReGPT/llama2-7b-phrase-tokenizer-trained-on-WikiText103/
	num_layers_unfrozen: -1
	gradient_checkpointing: True
	optimizer:
		name: Lamb
		kwargs:
			lr: 1e-4
			weight_decay: 0.0
			betas: [0.9, 0.999]
			eps: 1e-08
	scheduler:
		name: CosineAnnealingLR
		kwargs:
			T_max: 1000
			eta_min: 3e-5
	num_epochs: 1000
	log_with: tensorboard
	project_dir: output/
	negatives_in_device: True
	predict_from_last: 10000
	eval_step: 1000
	faiss:
		index_type: Flat
		dimension: 768
		nprobe: 10000
		phrases_path: ../data_of_ReGPT/phrases_WikiText-103/phrases.npy
		matrix_path: ../data_of_ReGPT/phrases_WikiText-103/phrases_embeddings.npy
	negative_path: ../data_of_ReGPT/phrases_WikiText-103/negati

In [4]:
print(generate('Deathly Hallows'))


<s> Deathly Hallows revolves largely around hercynicualism revolving around the Life and Death of two young young boys who each disguise themselves as innocents – who are lured to Otherworld around the world by stealing their  Precious treasureand Cyril . They are initially lured to other world world by manipulating them of his past life transform into Otherworld . Much like the Otherworld itself is structured by HerméticPhilosophies as elaborated by and Cyril themselves in the process evolve into a sentient spirit world that is based on the true nature of Hermiten angel .and Cyril calculated for full advisories on tropical systems at 210 to 230ze and intensified to attain wind speeds of 210km  Hlimestone-basalt mount .and Cyril were briefly bombarded by otherworld 2011 , it degenerated into a  Storm of Zdeath and destruction shortly afterwards by Damián Logan and Jeremy Scott who left alongside him afterwards to embark on his new voyage across the Atlantic to continent. In hopes of fi

In [5]:
start = time()
model.train_config['do_sample'] = True
model.train_config['top_k'] = 5
model.train_config['top_p'] = 0.8
model.train_config['max_length'] = 100
print(generate('Beijing is located in'))
print(time()-start)

<s> Beijing is located in Far Eastern of China at BaiXiau County with roughly ⅓ square meter common areas with roughly 240 of Square metres and about 260 of Square mètre des s of two sides. Facing North China clockwise from North China parallel between Beijing and Hangzhouprovince , while running parallel to eastern China – between the Far Eastern peninsula and the far eastern peninsula of North China mount .Wangchuck mount .Chanchu mount .Wangchuck mount .Wangchuck mount .Kangchờng mount .Kangchóng mount .Kangchóng mount .Kangchông mount .Kangchóng mount .Chanchu mount .Kangchongmount .Kangchoa mount .Kangchông mount .Chanchu mount .Chochangmount .Kangchong mount .Kangchong . Same name as a  Mountainmount .Huachung , known also English as  MountCha Chünnu ASTA EHünnu Asta Ehunnü stä
95.56739544868469


In [6]:
start = time()
model.train_config['do_sample'] = True
model.train_config['top_k'] = 5
model.train_config['top_p'] = 0.8
model.train_config['max_length'] = 100
print(generate('Tsinghua'))
print(time()-start)

<s> Tsinghua bei  Yang BeiYang Banquo is famed for including being chosen as one of the five leading heroines of East China in 2008 by the International Business Times named as one of 500 best female figures in the world by the Daily Globe .com named WuBao as one of the top five heroines of 2010s in 2010 by Business Times was described as the youngest person ever to win the Best Female Performance by a Female figure in the Worldlands of and South American Capistr O. Wu Tepisode , while praising her character and fearlessness in tandem with her fellow heroines garnered widespread acclaim from both critics and media critics . Upon its release , it  garnered widespread acclaim from critics and fans across both the mass media and mass media for both its portrayal and originality regards to its portrayal and Theatricality and Cyril Monteith were praised for their portrayal and artistry especially in regards to the fates of several older female characters in present day china . TE SingHua Be

In [7]:
start = time()
model.train_config['do_sample'] = True
model.train_config['top_k'] = 5
model.train_config['top_p'] = 0.8
model.train_config['max_length'] = 100
print(generate('Beijing'))
print(time()-start)

<s> Beijing City University bei Yang International University Bei Yang BeiYang International University bei and Hebei City University of bei  Yang Bei  Yang International University BeileYang International University West Point , ny. University BeiYangInternational University BeiYang BeiYangInternational University bei  Yang BeiYang i International BeiYang i International Bei yang International University bei yang i International Bei Yang international Bei rong i International Bei rong International Bei ronji IUgolinóInternational University BeiRonginternational bei rong i International Bei Rön JiUgolINOWorld University Bei Ron ji IUgolíno International University BeirongInternational Beirong international BeiRong i International bei Ron Ji Ugolíno World University bei ron JiUgolINOWorld University Bei 
88.25420546531677


In [8]:
start = time()
model.train_config['do_sample'] = True
model.train_config['top_k'] = 5
model.train_config['top_p'] = 0.8
model.train_config['max_length'] = 100
print(generate('Young lady loves me'))
print(time()-start)

<s> Young lady loves me Finally Parting with her child and second son loves .loving Family and  Friendship continues for more years until after she marries Annabel Finch . Elle soon becomes infatuated with her girlfriend and he had to leave her home for religious reasons for many years before Marrying Him . She again pursues  Himmlers who is now happily married again with his late mother Annabel  Finch. Betsy soon becomes romantically interested in Himmlers and soon becomes romantically involved with her stepbrother  Priscilla  Finch. Elle soon discovers that she is romantically interested in HimmLerloves .designation is given to her by & Mrs. Greene whom she originally met again during a fracas together while Annabel Joins Himmlerin her husband 's personal union . Ellen is initially reluctant design but hesitates again when she makes amends again when she makes plans to Marry  Himmtitanium , thus ending both her marriage and eventuality she is to revert to the previous life . She agai

In [10]:
start = time()
model.train_config['do_sample'] = True
model.train_config['top_k'] = 5
model.train_config['top_p'] = 0.8
model.train_config['max_length'] = 100
print(generate('Tell me a story about China'))
print(time()-start)

<s> Tell me a story about China . Leaving Home Front at home with family friend s in Portland , MainTheodore , Shelby lays Eyes Wide Shut  While on the road crew – and he is soon drawn back to Home Front while dealing with traumatic stress injuries suffered while returning from a boat trip . Back home ,  Shelby is soon welcomed back home by late friend  Priscilla Massey , whose daughter is being killed in 1918 flu the next month while at a work camp in Ben ItoTheodore , Shelby is soon welcomed back at home by family friend s in western  Willamette county . Back home , Priscilla sits and sleeps with of his friends again at Greenwood Memorial Church in Cedar bluff, where late mother  Priscilla 20 feet from her Honeymoon . Back home ,  Priscilla makes plans to return home as soon as she makes plans to Leave Home the next month alone after home with her late mother leaving .however she makes amends with her when she makes amends with her to former friend  Priscilla ascent , who now hate yo